
This notebook applies metadata Sheet 2 definitions to decode survey values.  
This notebook maps coded responses (e.g., 1, 2, 3) into human‑readable labels (e.g., Employed, Unemployed).

Dependencies:
- Run `00_Settings.ipynb` and `01_Inventory.ipynb` first.
- Requires outputs from `03_Metadata_Decoder.ipynb` (Header Encoded Surveys).
- Note: Survey CSVs already carry meaning from Sheet 2 reshaping. This notebook applies the final decoding logic.

Output:

- Fully decoded survey CSVs saved into **NEW Fully Decoded Surveys**.
- Reports per survey showing number of columns successfully decoded.

Notes:
- **Sheet 1 metadata** → header translation (column names).  
- **Sheet 2 metadata** → value translation (coded responses).  

**INTENT:** This notebook performs the **final decoding stage** of the Labor Force Survey pipeline.  
It applies **Sheet 2 metadata** to translate coded survey values into human‑readable labels.

- Next steps: duplicate variable detection, integrity checks, coverage scanning.


In [ ]:
# Linter stubs (will be overwritten when 00_Settings.ipynb runs)
BASE_PATH: str
inventory: dict

In [ ]:
# Ensure settings are loaded
%run ./00_Settings.ipynb
%run ./01_Inventory.ipynb

# Alias for compatibility
base_path = BASE_PATH

### Interim Sample 

#### Intent: Interim Sample Decoder

This interim function generates a **single sample output** (e.g., `NEW Fully Decoded Survey Sample` for **January 2018**) using the same decoding logic as the batch runner.  

Unlike the full batch process, which redirects all decoded surveys to Google Drive, this sample run saves directly into the local **interim repository path**.  

The purpose is to provide a quick preview of how a fully decoded CSV looks without requiring you to download the large, heavy files from Google Drive.  

Use this when:
- You want to validate decoding logic on a small subset before running the full batch.  
- You need a lightweight example for documentation, testing, or demonstration.  
- You want to inspect decoded values locally without waiting for the complete dataset.


In [ ]:

import os, re
import pandas as pd

# ------------------------------------------------------------
# Shared folder names 
# ------------------------------------------------------------
HEADER_ENCODED_FOLDER = "NEW Header Encoded Surveys"
FULLY_DECODED_FOLDER = "NEW Fully Decoded Surveys"
FULLY_DECODED_SAMPLE_FOLDER = "NEW Fully Decoded Survey Sample"

# ------------------------------------------------------------
# Helper: robust month parsing from filename
# ------------------------------------------------------------
MONTHS = [
    "JANUARY","FEBRUARY","MARCH","APRIL","MAY","JUNE",
    "JULY","AUGUST","SEPTEMBER","OCTOBER","NOVEMBER","DECEMBER"
]
MONTH_PATTERN = re.compile(r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)", re.IGNORECASE)


def load_clean_sheet2(base_path, year, month):
    """
    Load the Clean Sheet 2 Metadata for value decoding.
    """
    path = os.path.join(base_path, "Metadata Sheet 2 CSV's", year, f"Sheet2_{month}_{year}.csv")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Metadata not found at: {path}")
    return pd.read_csv(path, dtype=str)


def find_target_column(survey_columns, meta_desc):
    """
    Smart Matcher: Align metadata descriptions with survey columns.
    Handles cases like 'Highest Grade' vs 'C07-Highest Grade Completed'.
    """
    if pd.isna(meta_desc): return None
    meta_desc = str(meta_desc).strip()

    # 1. Exact Match
    if meta_desc in survey_columns: return meta_desc

    # 2. Metadata has prefix (Meta="C06-Status" -> Survey="Status")
    clean_meta = re.sub(r'^C\d+[\s\-_]+', '', meta_desc, flags=re.IGNORECASE).strip()
    if clean_meta in survey_columns: return clean_meta

    # 3. Survey has prefix (Meta="Status" -> Survey="C06-Status")
    for col in survey_columns:
        if col.endswith(meta_desc):
            prefix = col[:-len(meta_desc)].strip()
            if re.search(r'^C\d+[\s\-_]*$', prefix, re.IGNORECASE) or prefix == "":
                return col
    return None


def decode_survey_safe(survey_df, meta_df):
    """
    Decode survey values using metadata Sheet 2.
    Applies Smart Matcher and safe mapping logic.
    """
    unique_vars = meta_df['Variable'].unique()
    decoded_count = 0
    survey_cols = list(survey_df.columns)

    for var_code in unique_vars:
        subset = meta_df[meta_df['Variable'] == var_code].copy()
        if subset['Description'].isnull().all(): continue

        raw_desc = subset['Description'].dropna().iloc[0].strip()
        target_col = find_target_column(survey_cols, raw_desc)
        if not target_col: continue

        # Skip if all labels are zero/nan
        mask_zeros = subset['Label'].astype(str).isin(['0', '0.0', '0.00', 'nan', 'NaN'])
        if mask_zeros.all(): continue

        # Build lookup dictionary
        lookup = {}
        for _, row in subset.iterrows():
            try:
                label = row['Label']
                if str(label) in ['0', '0.0', 'nan']: continue

                min_v = float(row['min_value'])
                max_v = float(row['max_value'])

                if max_v > min_v and max_v != 0:
                    for c in range(int(min_v), int(max_v) + 1):
                        lookup[c] = label
                else:
                    lookup[int(min_v)] = label
            except:
                continue

        if not lookup: continue

        def safe_map(val):
            try: return lookup.get(int(float(val)), val)
            except: return val

        survey_df[target_col] = survey_df[target_col].apply(safe_map)
        decoded_count += 1

    return survey_df, decoded_count


# ============================================================
# Interim sample decoder (standalone)
# ============================================================
def run_sample_decoding(base_path,
                        year="2018",
                        month="January",
                        interim_root=r"C:\Users\SHANIA\Downloads\Preprocessing\Notebooks\data\interim"):
    """
    Decode a single survey file (e.g., January 2018) for demonstration.
    Saves into 'data/interim/NEW Fully Decoded Survey Sample/<year>/'.
    """
    # Normalize month capitalization (e.g., january -> January)
    month = month.strip().capitalize()
    if month.upper() not in MONTHS:
        raise ValueError(f"Invalid month: {month}. Expected one of: {', '.join(MONTHS)}")

    input_root = os.path.join(base_path, HEADER_ENCODED_FOLDER, year)
    output_root = os.path.join(interim_root, FULLY_DECODED_SAMPLE_FOLDER, year)
    os.makedirs(output_root, exist_ok=True)

    # Find the survey file for the given month
    if not os.path.exists(input_root):
        print(f"[SKIP] Input folder not found: {input_root}")
        return

    files = [f for f in os.listdir(input_root) if f.lower().endswith(".csv")]
    target_file = next((f for f in files if month.upper() in f.upper()), None)

    if not target_file:
        print(f"[SKIP] No survey file found for {month} {year} in {input_root}.")
        return

    print("================================================")
    print(f"SAMPLE DECODING: {month.upper()} {year}")
    print(f"Source: {input_root}")
    print(f"Dest:   {output_root}")
    print("================================================\n")

    try:
        # 1. Load Survey
        survey_path = os.path.join(input_root, target_file)
        df_survey = pd.read_csv(survey_path, low_memory=False)

        # 2. Load Metadata
        df_meta = load_clean_sheet2(base_path, year, month)

        # 3. Decode
        df_final, count = decode_survey_safe(df_survey, df_meta)

        # 4. Save
        save_path = os.path.join(output_root, target_file)
        df_final.to_csv(save_path, index=False)

        print(f"   [OK] Decoded {count} columns.")
        print(f"   [SAVED] {save_path}")

    except Exception as e:
        print(f"   [ERROR] {e}")


### Batch Runner

In [ ]:
# ============================================================
# Full batch decoder (unchanged, uses same helpers)
# ============================================================
def run_batch_decoding(base_path):
    """
    Batch decode all survey CSVs using Sheet 2 metadata.
    Saves results into 'NEW Fully Decoded Surveys'.
    """
    input_root = os.path.join(base_path, HEADER_ENCODED_FOLDER)
    output_root = os.path.join(base_path, FULLY_DECODED_FOLDER)
    os.makedirs(output_root, exist_ok=True)

    print("================================================")
    print("STARTING BATCH VALUE DECODING")
    print(f"Source: {input_root}")
    print(f"Dest:   {output_root}")
    print("================================================\n")

    if not os.path.exists(input_root):
        print(f"Error: Input folder not found: {input_root}")
        return

    success, errors = 0, 0

    year_folders = [f for f in os.listdir(input_root) if f.isdigit() and os.path.isdir(os.path.join(input_root, f))]
    for year in sorted(year_folders):
        year_in = os.path.join(input_root, year)
        year_out = os.path.join(output_root, year)
        os.makedirs(year_out, exist_ok=True)

        files = [f for f in os.listdir(year_in) if f.lower().endswith(".csv")]
        for filename in files:
            match = MONTH_PATTERN.search(filename)
            if not match: continue
            month = match.group(1).capitalize()

            print(f"Processing: {month.upper()} {year}...")

            try:
                df_survey = pd.read_csv(os.path.join(year_in, filename), low_memory=False)
                df_meta = load_clean_sheet2(base_path, year, month)
                df_final, count = decode_survey_safe(df_survey, df_meta)

                save_path = os.path.join(year_out, filename)
                df_final.to_csv(save_path, index=False)

                print(f"   [OK] Decoded {count} columns.")
                print(f"   [SAVED] {filename}")
                success += 1

            except FileNotFoundError as e:
                print(f"   [SKIP] Metadata missing: {e}")
            except Exception as e:
                print(f"   [ERROR] {e}")
                errors += 1

            print("-" * 40)

    print(f"\nCOMPLETED. Success: {success} | Errors: {errors}")
